In [2]:
import numpy as np

## Encoder: VGG16, Decoder: 

### Obtain Bottleneck Features

In [7]:
bottleneck_features = np.load('bottleneck_features/Flicker8k_bottleneck_features_VGG16_avgpooling.npz')
bottleneck_features_train = bottleneck_features["train"]
bottleneck_features_test = bottleneck_features["test"]
bottleneck_features_validation = bottleneck_features["validation"]

### Model Architecture

In [8]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

In [13]:
bottleneck_features_train.shape[1]

512

In [15]:
VGG16_model = Sequential()
VGG16_model.add(Dense(300, input_shape=(bottleneck_features_train.shape[1],)))
# Add Decoder
#
#
#
VGG16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 300)               153900    
Total params: 153,900
Trainable params: 153,900
Non-trainable params: 0
_________________________________________________________________


### Compile the Model

### Train the Model

In [29]:
from keras.callbacks import ModelCheckpoint

In [ ]:
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.VGG16.flicker8k', 
                               verbose=1, save_best_only=True)

# Model fit

### Load the Model with the Best Validation Loss

In [ ]:
VGG16_model.load_weights('saved_models/weights.best.VGG16.flicker8k')

### Test the Model